In [ ]:
import pandas as pd
import numpy as np
import json
from openpyxl import load_workbook, Workbook
import os

In [ ]:
## Assuming you have download Transnational list 2021-04-14.xlsx
## FILE: https://docs.google.com/spreadsheets/d/1zeWnuKsclWILw-LBltuE2q8glsjqg_vE/edit

In [ ]:
EXCEL_FILE = 'Transnational list 2021-04-14.xlsx'
EXCEL_TMP_FILE = '{}-tmp.xlsx'.format(EXCEL_FILE)
OUTPUT_FOLDER = "../backend/dev/resources/files/"
country_group_column = {0:'id',1:'label',2:'name'}

In [ ]:
def format_sheet(title):
    return 's-{}'.format(title.strip().replace(' ','').lower())

In [ ]:
## Create Temporary Clean Version of Excel Sheet
wb = load_workbook(EXCEL_FILE)
for a in wb:
    a.title = format_sheet(a.title)
wb.save(EXCEL_TMP_FILE)

In [ ]:
sheet_to_df_map = list(pd.read_excel(EXCEL_TMP_FILE, sheet_name=None))
country_group = pd.read_excel(EXCEL_TMP_FILE, sheet_name=sheet_to_df_map[0])
country_group = country_group[list(country_group)[:3]]
country_group_columns_rename = {}
for i, x in enumerate(list(country_group)[:3]):
    country_group = country_group.rename(columns={x:country_group_column[i]})

In [ ]:
country_group['id'] = country_group['id'].fillna(0).astype('int')
country_group['category'] = ''

In [ ]:
# indexing category and modify duplicated id
category = ''
current_id = country_group.loc[1,'id']
for i in range(0, len(country_group)):
    if country_group.loc[i, 'name'] != country_group.loc[i, 'name']:
        category = country_group.loc[i, 'label']
    if country_group.loc[i, 'name'] == country_group.loc[i, 'name']:
        country_group.loc[i,'category'] = category
        # remove below code if id is not duplicated anymore
        if current_id < int(country_group.loc[i,'id']):
            country_group.loc[i,'id'] = current_id + 1
        current_id = country_group.loc[i,'id']

In [ ]:
# Stip all extra spaces
object_column = list(country_group.select_dtypes(['object']))
country_group[object_column] = country_group[object_column].apply(lambda x: x.str.strip())

In [ ]:
country_group = country_group[country_group['name'] == country_group['name']].reset_index()
country_group['sheet_name'] = country_group['name'].apply(lambda x: format_sheet(x))
country_group['countries'] = ''

In [ ]:
# creating country_group_countries-new.json
# removing tmp file
country_group_countries = {}
for i in range(0, len(country_group)):
    sheet_name = country_group.loc[i,'sheet_name']
    country_group_name = country_group.loc[i,'name']
    try:
        countries = pd.read_excel(EXCEL_TMP_FILE, sheet_name=sheet_name, names=['name'])
        country_group_countries.update({country_group_name:list(countries['name'])})
    except:
        print('{} doesn\'t exists'.format(sheet_name))
        print('Trying to check matches with removing word "and"')
        try:
            countries = pd.read_excel(EXCEL_TMP_FILE, sheet_name=sheet_name.replace('and',''), names=['name'])
            country_group_countries.update({country_group_name:list(countries['name'])})
            print('{} Found'.format(sheet_name.replace('and','')))
        except:
            print('{} is still Not Found'.format(sheet_name.replace('and','')))

In [ ]:
OUTPUT_FILE = "{}country_group_countries-new.json".format(OUTPUT_FOLDER)

with open(OUTPUT_FILE, "w") as f: 
    json.dump(country_group_countries, f,indent=2)
    
os.remove(EXCEL_TMP_FILE)

In [ ]:
# We need to confirm to UNEP about type of country group
def get_country_group_type(x):
    return 'region' if 'region' in x.lower() else 'mea'
country_group['type'] = country_group['category'].apply(lambda x: get_country_group_type(x))

In [ ]:
columns = [country_group_column[x] for x in country_group_column]
columns.append('type')
columns.append('category')
country_group = country_group[columns]

In [ ]:
OUTPUT_FILE = "{}country_group-new.json".format(OUTPUT_FOLDER)
country_group.to_json(OUTPUT_FILE,orient='records',indent=2)